## Домашнее задание по первой лекции второго модуля.
### Практическая часть
**1. Симуляция секвенирования** (3 балла)  
Первое задание заключается в том, чтобы написать
симуляцию секвенирования ридов при помощи illumina.
Для этого необходимо сгенерировать случайно строку
длинной 50 000 символов, над алфавитом {A, T, G, C}.
После этого случайно выбирать подстроки, длинна
которых распределена нормально со средним 250 и
среднеквадратическим отклонением 30, каждую такую
подстроку "секвенировать".  
Под словом "секвенировать" в данном случае имеется
ввиду симуляция того процесса, который мы обсуждали
на лекции. Вы считываете очередной символ подстроки,
например, 100 раз, но в N случаях из 100 ошибаетесь
и считываете любой нуклеотид, кроме правильного с
одинаковой вероятностью. N принадлежит равномерному
распределению от 0 до 75. По получившемуся набору
вычисляете наибоее вероятный нуклеотид (тот, которого
больше всего) и его качество прочтения, чтобы
записать его в формате FASTQ.  
При симуляции важно запоминать ID рида и для каждого
ID позиции, в которых нуклеотид был считан неверно и
какой должен быть на самом деле (для этого просто
можно хранить 2 числа и помнить что у вас есть
исходная строка, откуда берутся риды). Это
понадобится для выполнения следующих заданий.  
Всего ридов пусть будет 50К.

In [2]:
import numpy as np

In [97]:
np.random.seed(0)
nucls = list('ATGC')

sequence = ''.join(np.random.choice(nucls, 50000))
simulation = dict()

for i, read_len in enumerate(np.random.normal(250, 30, 50000).astype(int)):
    read_id = np.random.randint(0, 50000 - read_len)
    read = list(sequence[read_id:read_id+read_len])
    mistakes_ids = []
    mistakes_ascii = []

    for nuc_ind, nuc in enumerate(read):
        n = np.random.randint(0, 75)
        mistakes = np.random.choice([x for x in nucls if x != nuc], n)
        vals, counts = np.unique(mistakes, return_counts=True)
        if n == 0:
            Q = 30
            mistakes_ascii.append(chr(Q + 33))
            continue
        max_wrong_ind = np.argmax(counts)
        if counts[max_wrong_ind] > 100 - n:
            read[nuc_ind] = vals[max_wrong_ind]
            Q = int(-10 * np.log10((100 - counts[max_wrong_ind]) / 100))
            mistakes_ids.append(nuc_ind)
        else:
            Q = int(-10 * np.log10(n / 100))
        mistakes_ascii.append(chr(Q + 33))
    if i % 5000 == 0:
        print(i)
    simulation[i] = [read_id, ''.join(read), mistakes_ids, ''.join(mistakes_ascii)]

0
5000
10000
15000
20000
25000
30000
35000
40000
45000


In [98]:
with open('reads.fasta', 'w') as file:
    for i, read_data in simulation.items():
        read_id, read, mistakes_ids, mistakes_ascii = read_data
        file.write(f'@SEQ_{i}_{read_id}\n')
        file.write(f'{read}\n')
        file.write('+\n')
        file.write(f'{mistakes_ascii}\n')

with open("sequense.fasta", "w") as file:
    file.write('@INITIAL_SEQ\n')
    file.write(sequence)

**2. Удаление ошибок (Trimmomatic)** (2 балла)
Получившиеся риды обработайте Trimmomatic со
стандартными параметрами с лекции. Пользуясь тем, что
вы знаете, для каждого рида ground truth, посчитайте,
в скольких случаях Trimmomatic удалил нуклеотиды,
которые были считаны верно, а в скольких действительно
удалил ошибочные.

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
!conda install -c bioconda fastqc -y
!pip install multiqc
! conda install -c bioconda trimmomatic -y

In [140]:
! trimmomatic SE -phred33 reads.fasta reads_trimmed.fasta ILLUMINACLIP:TruSeq3-SE:2:30:10 SLIDINGWINDOW:8:2 LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticSE: Started with arguments:
 -phred33 reads.fasta reads_trimmed.fasta ILLUMINACLIP:TruSeq3-SE:2:30:10 SLIDINGWINDOW:8:2 LEADING:3 TRAILING:3 MINLEN:36
Automatically using 2 threads
java.io.FileNotFoundException: /content/TruSeq3-SE (No such file or directory)
	at java.io.FileInputStream.open0(Native Method)
	at java.io.FileInputStream.open(FileInputStream.java:195)
	at java.io.FileInputStream.<init>(FileInputStream.java:138)
	at org.usadellab.trimmomatic.fasta.FastaParser.parse(FastaParser.java:54)
	at org.usadellab.trimmomatic.trim.IlluminaClippingTrimmer.loadSequences(IlluminaClippingTrimmer.java:110)
	at org.usadellab.trimmomatic.trim.IlluminaClippingTrimmer.makeIlluminaClippingTrimmer(IlluminaClippingTrimmer.java:71)
	at org.usadellab.trimmomatic.trim.TrimmerFactory.makeTrimmer(TrimmerFactory.java:32)
	at org.usadellab.trimmomatic.Trimmomatic.createTrimmers(Trimmomatic.java:59)
	at org.usadellab.trimmomatic.TrimmomaticSE.run(TrimmomaticSE.java:318)
	at org.usadellab.trim

In [141]:
inds = []
reads = []
with open('reads_trimmed.fasta', 'r') as file:
    while True:
        try:
            x = file.readline()
            inds.append(int(x.split('_')[1]))
            reads.append(file.readline())
            file.readline()
            file.readline()
        except:
            break
print(len(reads))

48458


In [142]:
right_del = 0
wrong_del = 0

for ind_read_trim, read_trim in zip(inds, reads):
    read = simulation[ind_read_trim][1]
    mistakes = simulation[ind_read_trim][2]
    for i, nuc in enumerate(read_trim):
        if i < len(read) and nuc != read[i]:
            if i in mistakes:
                right_del += 1
            else:
                wrong_del += 1

print(right_del, wrong_del)

81198 2575358



**3. Коррекция ошибок** (4 балла)
Получившиеся риды обработайте одним(любым) из инструментов, которые
использованы и проанализированны в
[этой статье](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-01988-3).
Посчитайте TP, FP, TN, FN.



In [104]:
! conda install -y lighter --channel bioconda

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::libgfortran-ng==7.5.0=ha8ba4b0_17
  - defaults/linux-64::hdf5==1.10.2=hba1933b_1
  - bioconda/linux-64::kallisto==0.44.0=h7d86c95_2
- \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: - \ | / - \ | / - \ | / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:


In [149]:
! lighter -r reads_trimmed.fasta -K 20 50000

[2024-06-22 03:04:30] =============Start====================
[2024-06-22 03:04:30] Scanning the input files to infer alpha(sampling rate)
[2024-06-22 03:04:30] Average coverage is 220.799 and alpha is 0.032
[2024-06-22 03:04:30] Bad quality threshold is "#"
[2024-06-22 03:04:31] Finish sampling kmers
[2024-06-22 03:04:31] Bloom filter A's false positive rate: 0.320316
[2024-06-22 03:04:31] The error rate is high. Lighter adjusts -maxcor to 5 and bad quality threshold to "$".
[2024-06-22 03:04:32] Finish storing trusted kmers
[2024-06-22 03:04:37] Finish error correction
Processed 48458 reads:
	1118 are error-free
	Corrected 182 bases(0.003845 corrections for reads with errors)
	Trimmed 0 reads with average trimmed bases 0.000000
	Discard 0 reads


In [152]:
tp = tn = fp = fn = 0
inds_cor = []
reads_cor = []
with open('reads_trimmed.cor.fq', 'r') as file:
    while True:
        try:
            x = file.readline()
            inds_cor.append(int(x.split('_')[1]))
            reads_cor.append(file.readline())
            file.readline()
            file.readline()
        except:
            break



print(len(inds), len(reads))
for i in range(len(inds)):
    start, read, mistakes_ids, _ = simulation[inds_cor[i]]
    for j, (nuc_cor, nuc_read, nuc_seq) in enumerate(zip(reads_cor[i], reads[i], sequence[start: start + len(read)])):
        if nuc_read != nuc_cor:
            if j in mistakes_ids and nuc_cor == nuc_seq:
                tp += 1
            elif j in mistakes_ids and nuc_cor != nuc_seq:
                fn += 1
            elif j not in mistakes_ids:
                fp += 1
        else:
            if j in mistakes_ids:
                fn += 1
            else:
                tn += 1

print(f"TP: {tp}")
print(f"FP: {fp}")
print(f"TN: {tn}")
print(f"FN: {fn}")

48458 48458
TP: 119
FP: 62
TN: 10739410
FN: 330306


In [155]:
import pickle

with open('saved_dictionary.pkl', 'wb') as f:
    pickle.dump(simulation, f)


**4. Сделайте какой-то вывод** (1 балл)
